<a href="https://colab.research.google.com/github/Deuce01/atlas/blob/main/Soccer_Prediction_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import requests
import datetime
import math
from scipy.stats import poisson
import firebase_admin
from firebase_admin import credentials, firestore

# --- CONFIGURATION ---
# We use Environment Variables for security (set these in GitHub later)
RAPIDAPI_KEY = os.environ.get("RAPIDAPI_KEY")
FIREBASE_KEY_JSON = os.environ.get("FIREBASE_KEY_JSON")

# API Endpoints
API_HOST = "api-football-v1.p.rapidapi.com"
HEADERS = {
    "X-RapidAPI-Key": RAPIDAPI_KEY,
    "X-RapidAPI-Host": API_HOST
}

# Top 5 Leagues IDs (to filter for high quality data)
# 39: Premier League, 140: La Liga, 78: Bundesliga, 135: Serie A, 61: Ligue 1
LEAGUE_IDS = [39, 140, 78, 135, 61]

# Initialize Firebase
if not firebase_admin._apps:
    cred_dict = json.loads(FIREBASE_KEY_JSON)
    cred = credentials.Certificate(cred_dict)
    firebase_admin.initialize_app(cred)

db = firestore.client()

# ==========================================
# 📡 PART 1: FETCH REAL DATA
# ==========================================

def get_fixtures_today():
    """Fetches all matches for the target leagues today"""
    today = datetime.date.today().strftime("%Y-%m-%d")
    url = f"https://{API_HOST}/v3/fixtures"

    # We fetch ALL matches for today (1 API Call)
    querystring = {"date": today, "status": "NS"} # NS = Not Started
    response = requests.get(url, headers=HEADERS, params=querystring)
    data = response.json()

    # Filter for our top leagues
    matches = [
        item for item in data.get('response', [])
        if item['league']['id'] in LEAGUE_IDS
    ]
    return matches

def get_team_stats(team_id, league_id, season):
    """
    Fetches standings to get Goals For/Against (1 API Call per league)
    Note: In a production app, cache this response to save API calls!
    """
    url = f"https://{API_HOST}/v3/standings"
    querystring = {"season": season, "league": league_id}
    response = requests.get(url, headers=HEADERS, params=querystring)
    data = response.json()

    try:
        standings = data['response'][0]['league']['standings'][0]
        # Find specific team stats
        for team in standings:
            if team['team']['id'] == team_id:
                return {
                    "played": team['all']['played'],
                    "goals_for": team['all']['goals']['for'],
                    "goals_against": team['all']['goals']['against']
                }
    except (IndexError, KeyError):
        return None
    return None

def get_league_averages(league_id, season):
    """
    Approximation of league averages.
    Ideally, calculate this from the full standings table.
    """
    # For the free script, we'll use static robust averages for Top 5 leagues
    # to save API calls (The free tier is 100 calls/day).
    return {"avg_goals_home": 1.55, "avg_goals_away": 1.25}

# ==========================================
# 🧠 PART 2: STATISTICAL ENGINE (POISSON)
# ==========================================

def calculate_probabilities(home_stats, away_stats, league_avg):
    if not home_stats or not away_stats: return None

    # Attack Strength (AS) & Defense Strength (DS)
    # Avoid division by zero
    home_games = max(home_stats['played'], 1)
    away_games = max(away_stats['played'], 1)

    has = (home_stats['goals_for'] / home_games) / league_avg['avg_goals_home']
    hds = (home_stats['goals_against'] / home_games) / league_avg['avg_goals_away']

    aas = (away_stats['goals_for'] / away_games) / league_avg['avg_goals_away']
    ads = (away_stats['goals_against'] / away_games) / league_avg['avg_goals_home']

    # Expected Goals (Lambda)
    lambda_home = has * ads * league_avg['avg_goals_home']
    lambda_away = aas * hds * league_avg['avg_goals_away']

    # Poisson Matrix
    prob_home_win, prob_draw, prob_away_win, prob_over_1_5 = 0, 0, 0, 0

    for h in range(6):
        for a in range(6):
            p = poisson.pmf(h, lambda_home) * poisson.pmf(a, lambda_away)
            if h > a: prob_home_win += p
            elif h == a: prob_draw += p
            elif a > h: prob_away_win += p
            if (h + a) > 1.5: prob_over_1_5 += p

    return {
        "home_win": prob_home_win,
        "draw": prob_draw,
        "away_win": prob_away_win,
        "double_chance_1x": prob_home_win + prob_draw,
        "over_1_5": prob_over_1_5,
        "lambda_total": lambda_home + lambda_away
    }

# ==========================================
# 🚀 PART 3: MAIN EXECUTION
# ==========================================

def run_analysis():
    print("⚽ Starting Analysis...")
    matches = get_fixtures_today()
    print(f"Found {len(matches)} matches in top leagues.")

    recommendations = []

    # Cache standings to avoid calling API multiple times for same league
    # (Simplified for this script)

    for fixture in matches:
        home_team = fixture['teams']['home']
        away_team = fixture['teams']['away']
        league_id = fixture['league']['id']
        season = fixture['league']['season']

        # 1. Get Stats (Real Data)
        # Note: This consumes 2 calls per match.
        # CAUTION: On free tier, limit this to the first 5 matches or implement caching.
        if len(recommendations) >= 5: break

        h_stats = get_team_stats(home_team['id'], league_id, season)
        a_stats = get_team_stats(away_team['id'], league_id, season)

        # 2. Run Algorithm
        if h_stats and a_stats:
            avgs = get_league_averages(league_id, season)
            probs = calculate_probabilities(h_stats, a_stats, avgs)

            # 3. Filter for "Bankers" (High Probability)

            # Market: Over 1.5 Goals
            if probs['over_1_5'] > 0.78:
                recommendations.append({
                    "match": f"{home_team['name']} vs {away_team['name']}",
                    "market": "Over 1.5 Goals",
                    "odds": 1.30, # In real app, fetch odds from 'fixture' object if available
                    "confidence": int(probs['over_1_5'] * 100),
                    "logic": f"Exp Goals: {probs['lambda_total']:.2f}"
                })

            # Market: Home Win
            elif probs['home_win'] > 0.65:
                recommendations.append({
                    "match": f"{home_team['name']} vs {away_team['name']}",
                    "market": "Home Win",
                    "odds": 1.45,
                    "confidence": int(probs['home_win'] * 100),
                    "logic": f"Home Advantage: {int(probs['home_win']*100)}%"
                })

    # Select Top 3
    recommendations.sort(key=lambda x: x['confidence'], reverse=True)
    top_picks = recommendations[:3]

    # Upload to Firestore
    if top_picks:
        total_odds = 1.0
        for p in top_picks: total_odds *= p['odds']

        doc_data = {
            "date": datetime.date.today().strftime("%Y-%m-%d"),
            "created_at": firestore.SERVER_TIMESTAMP,
            "total_odds": round(total_odds, 2),
            "status": "pending",
            "matches": top_picks
        }

        today_str = datetime.date.today().strftime("%Y-%m-%d")
        db.collection('daily_predictions').document(today_str).set(doc_data)
        print(f"✅ Uploaded {len(top_picks)} picks to Firestore.")
    else:
        print("❌ No high-confidence games found today.")

if __name__ == "__main__":
    run_analysis()